In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE110298"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE110298"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE110298.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE110298.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE110298.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I understand the task requires creating and analyzing a clinical dataset from the provided sample characteristics dictionary. Here's my implementation:

```python
import pandas as pd
import os
import numpy as np
import re

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray data which indicates gene expression
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Depression), looking at row 6
trait_row = 6  # Depression scores are available in row 6

# For age, looking at row 2
age_row = 2  # Age is available in row 2

# For gender, looking at row 1
gender_row = 1  # Gender is available in row 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert depression value to binary (0/1)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract numerical value after the colon
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        depression_score = int(value)
        # Convert to binary: 0 for no depression (score 0), 1 for any depression symptoms (score > 0)
        return 0 if depression_score == 0 else 1
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Convert age to continuous value"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract numerical value after the colon
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after the colon
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical data from the sample characteristics dictionary
    # Sample data from the provided dictionary
    sample_characteristics = {
        0: ['tissue: hippocampus'], 
        1: ['sex (self-reported): female', 'sex (self-reported): male'], 
        2: ['age: 89', 'age: 95', 'age: 84', 'age: 76', 'age: 86', 'age: 96', 'age: 80', 'age: 101', 'age: 85', 'age: 92', 'age: 78', 'age: 88', 'age: 81', 'age: 91', 'age: 99', 'age: 79', 'age: 93', 'age: 87'], 
        3: ['physical activity tier: low activity', 'physical activity tier: moderate activity', 'physical activity tier: high activity'], 
        4: ['physical activity: Actical average counts/day: 1.02', 'physical activity: Actical average counts/day: 0.74', 'physical activity: Actical average counts/day: 1.05', 'physical activity: Actical average counts/day: 0.71', 'physical activity: Actical average counts/day: 0.13', 'physical activity: Actical average counts/day: 1.06', 'physical activity: Actical average counts/day: 0.37', 'physical activity: Actical average counts/day: 0.89', 'physical activity: Actical average counts/day: 0.94', 'physical activity: Actical average counts/day: 0.65', 'physical activity: Actical average counts/day: 0.57', 'physical activity: Actical average counts/day: 1.75', 'physical activity: Actical average counts/day: 2', 'physical activity: Actical average counts/day: 1.48', 'physical activity: Actical average counts/day: 1.72', 'physical activity: Actical average counts/day: 2.06', 'physical activity: Actical average counts/day: 1.4', 'physical activity: Actical average counts/day: 1.8', 'physical activity: Actical average counts/day: 1.99', 'physical activity: Actical average counts/day: 1.87', 'physical activity: Actical average counts/day: 1.44', 'physical activity: Actical average counts/day: 2.32', 'physical activity: Actical average counts/day: 2.6', 'physical activity: Actical average counts/day: 3.53', 'physical activity: Actical average counts/day: 2.82', 'physical activity: Actical average counts/day: 4.19', 'physical activity: Actical average counts/day: 5.6', 'physical activity: Actical average counts/day: 3.26', 'physical activity: Actical average counts/day: 2.99', 'physical activity: Actical average counts/day: 4.7'], 
        5: ['education: 10', 'education: 14', 'education: 16', 'education: 12', 'education: 13', 'education: 21', 'education: 11', 'education: 18', 'education: 20', 'education: 15', 'education: 19', 'education: 22'], 
        6: ['depression: 0', 'depression: 2', 'depression: 8', 'depression: 1', 'depression: 3', 'depression: 4'], 
        7: ['cognitive frequency: 2.429', 'cognitive frequency: 3.143', 'cognitive frequency: 3', 'cognitive frequency: 3.714', 'cognitive frequency: NA', 'cognitive frequency: 1', 'cognitive frequency: 1.14', 'cognitive frequency: 2.29', 'cognitive frequency: 2.57', 'cognitive frequency: 3.57', 'cognitive frequency: 3.71', 'cognitive frequency: 3.571', 'cognitive frequency: 3.86', 'cognitive frequency: 2.71', 'cognitive frequency: 4.29', 'cognitive frequency: 3.429', 'cognitive frequency: 3.857', 'cognitive frequency: 4.143', 'cognitive frequency: 2.286', 'cognitive frequency: 2.86', 'cognitive frequency: 4', 'cognitive frequency: 2.43'], 
        8: ['social frequency: 2.167', 'social frequency: 1.333', 'social frequency: 2.333', 'social frequency: 3', 'social frequency: NA', 'social frequency: 1.83', 'social frequency: 2.83', 'social frequency: 2.17', 'social frequency: 1.5', 'social frequency: 2.33', 'social frequency: 2.5', 'social frequency: 1.833', 'social frequency: 2', 'social frequency: 2.67', 'social frequency: 3.167', 'social frequency: 3.333', 'social frequency: 2.667', 'social frequency: 1.67']
    }
    
    # Convert the sample characteristics dictionary to a DataFrame format expected by geo_select_clinical_features
    # We want each row to represent a characteristic and each column to represent a sample
    # First, determine


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# Since we don't have output from a previous step, let's first check if the files exist
# before making assumptions about the data

# Check for matrix file which typically contains sample information
matrix_files = [f for f in os.listdir(in_cohort_dir) if f.endswith("_matrix.txt")]
sample_files = [f for f in os.listdir(in_cohort_dir) if "sample" in f.lower() and f.endswith(".txt")]
clinical_files = [f for f in os.listdir(in_cohort_dir) if "clinical" in f.lower()]

# Print available files to understand what we're working with
print(f"Files in {in_cohort_dir}:")
for file in os.listdir(in_cohort_dir):
    print(f"  - {file}")

# 1. Determine gene expression data availability
# For GEO datasets, we typically look for series matrix files which contain gene expression data
# Let's check if any files suggest gene expression data
is_gene_available = any(f for f in os.listdir(in_cohort_dir) if "matrix" in f.lower() or "expression" in f.lower())
print(f"Gene expression data available: {is_gene_available}")

# 2. Variable Availability - Since we don't have sample characteristics data yet, we'll set these to None
# We'll need to analyze the actual data in future steps
trait_row = None
age_row = None
gender_row = None

# Define conversion functions that will be used if we find the relevant data
def convert_trait(value: str) -> int:
    """Convert depression status to binary (0: Control, 1: Depression)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Convert to binary
    if 'depression' in value or 'mdd' in value or 'case' in value or 'patient' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age to float."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    # Try to extract numeric value
    try:
        # Handle ranges (e.g., "25-30") by taking average
        if '-' in value:
            parts = value.split('-')
            return (float(parts[0]) + float(parts[1])) / 2
        
        # Handle years notation
        if 'years' in value or 'y' in value:
            return float(value.replace('years', '').replace('y', '').strip())
        
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """Convert gender to binary (0: Female, 1: Male)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    # Convert to binary
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata with initial filtering
# As we don't have enough information yet, we'll set is_trait_available based on trait_row
is_trait_available = trait_row is not None
print(f"Trait data available: {is_trait_available}")

# Validate and save cohort information with is_final=False for initial validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Skip clinical feature extraction for now since we don't have confirmed data availability
print("Clinical feature extraction will be performed in a later step once data availability is confirmed.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# These identifiers appear to be probe IDs from Affymetrix microarray platform
# Format like "1007_s_at" is typical of Affymetrix array probes
# These are not standard human gene symbols (like BRCA1, TP53, etc.)
# Therefore, they need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look at the columns available and choose the correct ones for mapping
# Based on the columns in the preview, 'ID' is for probe IDs and 'Gene Symbol' contains gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'
print(f"\nWill use '{prob_col}' for probe IDs and '{gene_col}' for gene symbols in mapping step")

# Test extracting mapping data
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"\nMapping data shape: {mapping_data.shape}")
print("First few rows of mapping data:")
print(preview_df(mapping_data, n=3))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Load the gene expression data again
genetic_data = get_genetic_data(matrix_file)
print(f"Loaded genetic data with shape: {genetic_data.shape}")

# 3. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)
print(f"Loaded gene annotation data with shape: {gene_annotation.shape}")

# 4. Perform gene identifier mapping
print("\nPerforming gene identifier mapping...")

# Create mapping from probe IDs to gene symbols
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Created mapping data with shape: {mapping_data.shape}")
print(f"Sample of mapping data: {preview_df(mapping_data, n=3)}")

# Apply the mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(expression_df=genetic_data, mapping_df=mapping_data)
print(f"Transformed gene data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Preview a sample of the mapped gene expression data
gene_sample = gene_data.iloc[:5, :5]
print("\nSample of gene expression data (first 5 genes, first 5 samples):")
print(gene_sample)


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Re-extract gene expression data and create the gene_data
# Load the gene expression data
genetic_data = get_genetic_data(matrix_file)
gene_annotation = get_gene_annotation(soft_file)
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
gene_data = apply_gene_mapping(expression_df=genetic_data, mapping_df=mapping_data)

# Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Re-extract the clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define conversion functions for clinical features
def convert_trait(value):
    """Convert depression score to binary format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip()
    try:
        # Depression is measured as a score, converting to binary
        # 0 = no depression, 1 = any depression symptoms
        depression_score = int(value)
        return 0 if depression_score == 0 else 1
    except:
        return None

def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# Get clinical data using the identified row indices
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=6,  # Depression scores in row 6
    convert_trait=convert_trait,
    age_row=2,    # Age data in row 2
    convert_age=convert_age,
    gender_row=1, # Gender data in row 1
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from hippocampus samples of elderly subjects with depression scores."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")